# Colab code

In [1]:
if 'google.colab' in str(get_ipython()):
  !apt install subversion
  !svn checkout https://github.com/sbettid/ATML_Project_202021
  # connect to the datasets
  closed_eyes_dataset_path = "/content/ATML_Project_202021/trunk/mrlEyes_2018_01"
else:
  closed_eyes_dataset_path = "mrlEyes_2018_01"

Output streaming troncato alle ultime 5000 righe.
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01330_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01331_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01333_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01334_1_0_1_0_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01335_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01345_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01351_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01352_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01354_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01356_1_0_1_2_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/val/1/s0031_01358_1_0_1_0_1_02.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/v

# Import required libraries

In [2]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

## Reading the dataset and splitting

In [3]:
def color_to_3gray(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img2 = np.zeros_like(img)
  img2[:,:,0] = gray
  img2[:,:,1] = gray
  img2[:,:,2] = gray

  return img2

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=color_to_3gray, horizontal_flip=True, brightness_range=(0.2,0.8), zoom_range=(0.3,0.7))
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5, preprocessing_function=color_to_3gray, horizontal_flip=True, brightness_range=(0.2,0.8), zoom_range=(0.3,0.7))

In [6]:
train_generator = train_datagen.flow_from_directory(closed_eyes_dataset_path + "/train/",target_size=(80,80), class_mode="binary")

Found 59428 images belonging to 2 classes.


In [7]:
val_generator = val_datagen.flow_from_directory(closed_eyes_dataset_path + "/val/",target_size=(80,80), subset="training", class_mode="binary")
test_generator = val_datagen.flow_from_directory(closed_eyes_dataset_path + "/val/",target_size=(80,80), subset="validation", class_mode="binary")

Found 12735 images belonging to 2 classes.
Found 12735 images belonging to 2 classes.


## Importing pre-trained model

In [8]:
from tensorflow.keras.applications.xception import Xception

In [9]:
xmodel = Xception(input_shape = (80,80,3), include_top = False, weights = "imagenet", pooling="max")

83689472/83683744 [==============================] - 1s 0us/step


In [10]:
layers_num = len(xmodel.layers)

In [11]:
i = 1

for layer in xmodel.layers:
    if layers_num - i <= 5:
        layer.trainable = True
    else:
        layer.trainable = False
    i += 1

In [12]:
xmodel.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 39, 39, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 39, 39, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 39, 39, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [13]:
last_layer = xmodel.get_layer('global_max_pooling2d')
last_output = last_layer.output

our_layer = keras.layers.Flatten()(last_output)
our_layer = keras.layers.Dense(512, activation='relu')(our_layer)
our_layer = keras.layers.Dropout(0.5)(our_layer)
our_layer = keras.layers.Dense(256, activation='relu')(our_layer)
our_layer = keras.layers.Dropout(0.3)(our_layer)
our_layer = keras.layers.Dense(1, activation='sigmoid')(our_layer)

In [14]:
model = keras.Model(xmodel.input,our_layer)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 39, 39, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 39, 39, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 39, 39, 32)   0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [16]:
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [17]:
history = model.fit(train_generator, validation_data=val_generator, epochs = 5)

Epoch 1/5
1858/1858 [==============================] - 184s 94ms/step - loss: 0.1964 - accuracy: 0.9227 - val_loss: 0.1229 - val_accuracy: 0.9544
Epoch 2/5
1858/1858 [==============================] - 173s 93ms/step - loss: 0.1133 - accuracy: 0.9579 - val_loss: 0.1201 - val_accuracy: 0.9600
Epoch 3/5
1858/1858 [==============================] - 173s 93ms/step - loss: 0.1029 - accuracy: 0.9628 - val_loss: 0.1018 - val_accuracy: 0.9611
Epoch 4/5
1858/1858 [==============================] - 173s 93ms/step - loss: 0.0950 - accuracy: 0.9658 - val_loss: 0.0915 - val_accuracy: 0.9662
Epoch 5/5
1858/1858 [==============================] - 172s 93ms/step - loss: 0.0861 - accuracy: 0.9691 - val_loss: 0.0870 - val_accuracy: 0.9676


## Checking results on the test set

In [18]:
model.evaluate(test_generator)

398/398 [==============================] - 30s 77ms/step - loss: 0.0688 - accuracy: 0.9775


[0.0687674731016159, 0.9775422215461731]

## Saving the model

In [19]:
model.save("model.h5")